### Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
- Наименование вакансии.
- Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
- Ссылку на саму вакансию.
- Сайт, откуда собрана вакансия. (можно прописать статично hh.ru или superjob.ru)

#### Import all the necessary libs:

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlencode
from datetime import datetime

#### Job title constant for hh.ru scraping (or can be arg in .py script):

In [2]:
job_title = "Data engineer" 

#### Function that returns an appropriate url link for scraping:

In [3]:
def prepare_url(job_title, page=0):
    #job_title = "Data engineer"
    url = "https://hh.ru/search/vacancy"
    params = {
    "text": job_title,
    "from":"suggest_post",
    "clusters":"true",
    "ored_clusters":"true",
    "enable_snippets":"true",
    "page":page}
    params_encoded = urlencode(params)
    return f"{url}?{params_encoded}"

    

#### Function that returns a DOM object by going to a specific page using generic url link:

In [4]:
def get_dom_from_url(job_title, session, page = 0):
    headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
    }
    response = session.get(prepare_url(job_title, page), headers=headers)
    return BeautifulSoup(response.text, "html.parser")


#### Function that returns an integer value of maximum pages for current "job_title" url search:

In [5]:
def get_search_max_pages(job_title, session):
    response = get_dom_from_url(job_title, session)
    return int(response.find("a",{"class":"bloko-button",
                              "data-qa":"pager-next" }).previous_sibling.find('a',{"data-qa":"pager-page"}).text)
    

#### Function that scraps through the vacancies on the page and writes it into the vacancy_list:

In [6]:
def scrap_page(vacancies, vacancy_list):
    for vacancy in vacancies:
            vacancy_info = {}
            vacancy_data = vacancy.find("div", {"class":""})
            vacancy_titile = vacancy_data.find("a", {"data-qa":"vacancy-serp__vacancy-title"}).text
            vacancy_link = vacancy_data.find("a", {"data-qa":"vacancy-serp__vacancy-title"}).get("href")
            if vacancy_data.find("span", {"data-qa":"vacancy-serp__vacancy-compensation"}) == None: # if salary is None then hardcode None
                vacancy_salary = None
                salary_from = None
                salary_to = None
                salary_currency = None
            else:
                vacancy_salary = vacancy_data.find("span", {"data-qa":"vacancy-serp__vacancy-compensation"}).text
                salary_currency = vacancy_salary.split(" ")[-1].replace(".","")
                if ("от" in vacancy_salary):
                    salary_from = int(vacancy_salary.split(" ")[1].replace("\u202f", ""))
                    salary_to = None
                elif "до" in vacancy_salary:
                    salary_from = None
                    salary_to = int(vacancy_salary.split(" ")[1].replace("\u202f", ""))
                elif "–" in vacancy_salary:
                    salary_from = int(vacancy_salary.split(" ")[0].replace("\u202f", ""))
                    salary_to = int(vacancy_salary.split(" ")[2].replace("\u202f", ""))
                else:
                    salary_from = None
                    salary_to = None
            vacancy_info["vacancy_titile"] = vacancy_titile
            vacancy_info["salary_from"] = salary_from
            vacancy_info["salary_to"] = salary_to
            vacancy_info["salary_currency"] = salary_currency
            vacancy_info["vacancy_link"] = vacancy_link
            vacancy_info["site_from"] = "hh.ru"
            vacancy_list.append(vacancy_info)

    

#### Function that iterates through the available pages and runs the "scrap_page()" function for each applicable url page:

In [7]:
def scrap_pages(job_title, session, vacancy_list):
    max_pages = get_search_max_pages(job_title, session)
    print(f"Starting scraping through pages for {job_title} position")
    for page in range(max_pages):
        print(f"Scraping page {page+1} out of {max_pages}...")
        page_dom = get_dom_from_url(job_title, session, page)
        vacancies = page_dom.find_all("div", {"class":"vacancy-serp-item-body__main-info"})
        scrap_page(vacancies, vacancy_list)
    print(f"Scraping is finished, found total of {len(vacancy_list)} vacancies")
        
        
        

#### Function that writes all the scraped data throught pages into one .csv file:

In [8]:
def write_to_csv(job_title, vacancy_list):
    df = pd.DataFrame(vacancy_list)
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"{current_time}_{job_title}_HH_Vacancies.csv"
    df.to_csv(filename, sep=';', float_format='%.2f', index=False)
    print(f'File "{filename}" successfully generated')

#### Main function:

In [9]:
def main(job_title):
    vacancy_list = []
    session = requests.Session()
    scrap_pages(job_title, session, vacancy_list)
    write_to_csv(job_title, vacancy_list)

In [10]:
main(job_title)

Starting scraping through pages for Data engineer position
Scraping page 1 out of 31...
Scraping page 2 out of 31...
Scraping page 3 out of 31...
Scraping page 4 out of 31...
Scraping page 5 out of 31...
Scraping page 6 out of 31...
Scraping page 7 out of 31...
Scraping page 8 out of 31...
Scraping page 9 out of 31...
Scraping page 10 out of 31...
Scraping page 11 out of 31...
Scraping page 12 out of 31...
Scraping page 13 out of 31...
Scraping page 14 out of 31...
Scraping page 15 out of 31...
Scraping page 16 out of 31...
Scraping page 17 out of 31...
Scraping page 18 out of 31...
Scraping page 19 out of 31...
Scraping page 20 out of 31...
Scraping page 21 out of 31...
Scraping page 22 out of 31...
Scraping page 23 out of 31...
Scraping page 24 out of 31...
Scraping page 25 out of 31...
Scraping page 26 out of 31...
Scraping page 27 out of 31...
Scraping page 28 out of 31...
Scraping page 29 out of 31...
Scraping page 30 out of 31...
Scraping page 31 out of 31...
Scraping is finished